In [3]:
!pip install rasa==1.10.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
 !pip install -U tables==3.6.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
 !pip install -U ipython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!python -m spacy download en

2022-07-05 14:31:41.950877: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-07-05 14:31:41.951075: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-07-05 14:31:41.951096: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://p

In [7]:
!pip install nest_asyncio==1.3.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import os
import rasa
import nest_asyncio

nest_asyncio.apply()

print("event loop ready")

event loop ready


In [9]:
from rasa.cli.scaffold import create_initial_project

In [10]:
project = 'test_project'
create_initial_project(project)

In [11]:
os.chdir(project)
print(os.listdir("."))

['domain.yml', '__init__.py', 'config.yml', 'endpoints.yml', 'models', '__pycache__', 'data', 'actions.py', 'test_project', 'credentials.yml', 'tests']


In [12]:
config = "config.yml"
training_files = "data/"
domain = 'domain.yml'
output = 'models/'

print(config, training_files, domain, output)

config.yml data/ domain.yml models/


In [13]:
model_path = rasa.train(domain, config, [training_files], output)
print(model_path)

Training Core model...


Processed trackers: 100%|██████████| 5/5 [00:00<00:00, 109.28it/s, # actions=16]
Processed actions: 16it [00:00, 617.23it/s, # examples=16]
Epochs: 100%|██████████| 100/100 [00:18<00:00,  5.49it/s, t_loss=0.076, loss=0.008, acc=1.000]


Core model training completed.
Training NLU model...


/usr/local/lib/python3.7/dist-packages/rasa/utils/common.py:363: UserWarning: You specified 'DIET' to train entities, but no entities are present in the training data. Skip training of entities.
Epochs: 100%|██████████| 100/100 [00:18<00:00,  5.54it/s, t_loss=1.602, i_loss=0.215, i_acc=1.000]


NLU model training completed.
Your Rasa model is trained and saved at '/content/test_project/models/20220705-143336.tar.gz'.
models/20220705-143336.tar.gz


In [14]:
from rasa.jupyter import chat

endpoints = 'endpoints.yml'

chat(model_path, endpoints)

Your bot is ready to talk! Type your messages here or send '/stop'.
hey
Hey! How are you?
principal
Great, carry on!
/stop


In [20]:
%%writefile data/nlu.md 
## intent:principal_name
- who is the principal
- principal
- principal's name
- can i know principal name
- can i know principal's name

## intent:location
- where is it located
- location
- where is it situated
- college address

## intent:greet
- hey
- hello
- hi
- good morning
- good evening
- hey there

## intent:goodbye
- bye
- goodbye
- see you around
- see you later

## intent:affirm
- yes
- indeed
- of course
- that sounds good
- correct

## intent:deny
- no
- never
- I don't think so
- don't like that
- no way
- not really

## intent:mood_great
- perfect
- very good
- great
- amazing
- wonderful
- I am feeling very good
- I am great
- I'm good

## intent:mood_unhappy
- sad
- very sad
- unhappy
- bad
- very bad
- awful
- terrible
- not very good
- extremely sad
- so sad

## intent:bot_challenge
- are you a bot?
- are you a human?
- am I talking to a bot?
- am I talking to a human?


Overwriting data/nlu.md


In [21]:
%%writefile domain.yml 

intents:
  - greet
  - goodbye
  - affirm
  - deny
  - mood_great
  - mood_unhappy
  - bot_challenge
  - principal_name
  - location

responses:
  utter_principal_name:
  - text : "His name is Dr. Mohan Babu"

  utter_location:
  - text : "Address : BMS Institute of Technology and Management,
Doddaballapur Main Road,
Avalahalli, Yelahanka,
Bengaluru - 560064."

  utter_greet:
  - text: "Hey! How are you?"

  utter_cheer_up:
  - text: "Here is something to cheer you up:"
    image: "https://i.imgur.com/nGF1K8f.jpg"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_happy:
  - text: "Great, carry on!"

  utter_goodbye:
  - text: "Bye"

  utter_iamabot:
  - text: "I am a bot, powered by Rasa."

session_config:
  session_expiration_time: 60
  carry_over_slots_to_new_session: true


Overwriting domain.yml


In [22]:
%%writefile data/stories.md 
## happy path
* greet
  - utter_greet
* mood_great
  - utter_happy
 
## principal
* greet
  - utter_greet
* principal_name
  - utter_principal_name

## location
*greet
  -utter_greet
*location
  - utter_location

## sad path 1
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* affirm
  - utter_happy

## sad path 2
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* deny
  - utter_goodbye

## say goodbye
* goodbye
  - utter_goodbye

## bot challenge
* bot_challenge
  - utter_iamabot


Overwriting data/stories.md


In [23]:
model_path = rasa.train(domain, config, [training_files], output)
print(model_path)

Processed Story Blocks:   0%|          | 0/7 [00:00<?, ?it/s, # trackers=24]

Training Core model...


Processed trackers: 100%|██████████| 7/7 [00:00<00:00, 105.96it/s, # actions=20]
Processed actions: 20it [00:00, 527.76it/s, # examples=20]
Epochs: 100%|██████████| 100/100 [00:23<00:00,  4.22it/s, t_loss=0.079, loss=0.002, acc=1.000]


Core model training completed.
Training NLU model...


/usr/local/lib/python3.7/dist-packages/rasa/utils/common.py:363: UserWarning: You specified 'DIET' to train entities, but no entities are present in the training data. Skip training of entities.
Epochs: 100%|██████████| 100/100 [00:19<00:00,  5.08it/s, t_loss=1.860, i_loss=0.376, i_acc=1.000]


NLU model training completed.
Your Rasa model is trained and saved at '/content/test_project/models/20220705-144323.tar.gz'.
models/20220705-144323.tar.gz


In [24]:
endpoints = "endpoints.yml"
chat(model_path, endpoints)

Your bot is ready to talk! Type your messages here or send '/stop'.
hi
Hey! How are you?
principal
His name is Dr. Mohan Babu
location
Address : BMS Institute of Technology and Management, Doddaballapur Main Road, Avalahalli, Yelahanka, Bengaluru - 560064.
principal
His name is Dr. Mohan Babu
can i know principal name
Address : BMS Institute of Technology and Management, Doddaballapur Main Road, Avalahalli, Yelahanka, Bengaluru - 560064.
can i know principal's name
Address : BMS Institute of Technology and Management, Doddaballapur Main Road, Avalahalli, Yelahanka, Bengaluru - 560064.
principal's name
His name is Dr. Mohan Babu
/stop
